<a href="https://colab.research.google.com/github/haydenlee914/Sports-analysis/blob/main/Sports_Data_Analysis_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# First, let's load the uploaded dataset to understand its structure and contents.
import pandas as pd

# Load the dataset
data = pd.read_csv('C:/Users/heyri/OneDrive/Desktop/data_sports.csv')

# Display the first few rows of the dataframe to understand its structure
data.head()

,Name,2023 Payroll Dollars,G_B,PA_B,HR,RBI,OPS,G_F,IP_F,Fld%,Rtot,Rdrs
0,Christian Walker,"$6,500,000",157,661,33,103,0.830,152,1309.1,0.998,-2,9
1,Matt Olson,"$21,000,000",162,720,54,139,0.993,162,1426.0,0.994,6,8
2,Ryan OHearn,"$1,400,000",112,368,14,60,0.801,70,519.0,0.994,-2,-5
3,Ryan Mountcastle,"$738,400",115,470,18,68,0.779,90,769.1,0.997,6,2
4,Bobby Dalbec,"$720,000",21,53,1,1,0.570,15,98.0,0.990,3,0


In [ ]:
# Normalize specified columns
columns_to_normalize = ['HR', 'RBI', 'OPS', 'Fld%', 'Rtot', 'Rdrs']
for col in columns_to_normalize:
    min_val = data[col].min()
    max_val = data[col].max()
    data[f'{col}_norm'] = (data[col] - min_val) / (max_val - min_val)

# Calculate additional normalized values needed for OE and DE
data['PA_B/G_B_norm'] = (data['PA_B'] / data['G_B'] - (data['PA_B'] / data['G_B']).min()) / ((data['PA_B'] / data['G_B']).max() - (data['PA_B'] / data['G_B']).min())
data['IP_F/G_F_norm'] = (data['IP_F'] / data['G_F'] - (data['IP_F'] / data['G_F']).min()) / ((data['IP_F'] / data['G_F']).max() - (data['IP_F'] / data['G_F']).min())


# Step 1: Calculate OE and DE

# Calculate Offensive Efficiency (OE)
data['OE'] = 0.4 * data['OPS_norm'] + 0.2 * data['HR_norm'] + 0.2 * data['RBI_norm'] + 0.2 * data['PA_B/G_B_norm']

# Calculate Defensive Efficiency (DE)
data['DE'] = 0.4 * data['Fld%_norm'] + 0.3 * data['Rdrs_norm'] + 0.2 * data['Rtot_norm'] + 0.1 * data['IP_F/G_F_norm']
# Fill NaN values in 'DE' column with 0
data['DE'] = data['DE'].fillna(0)

In [ ]:
# Step 2: Rank Teams

# Normalize the payroll salary
data['2023 Payroll Dollars'] = data['2023 Payroll Dollars'].replace('[\$,]', '', regex=True).astype(float)

data['Payroll Rank'] = data['2023 Payroll Dollars'].rank(method='min')
data['Performance Rank'] = (data['OE'] + data['DE']).rank(ascending=False, method='min')

# Convert 'Payroll Rank' and 'Performance Rank' to integer to remove decimal points
data['Payroll Rank'] = data['Payroll Rank'].astype(int)
data['Performance Rank'] = data['Performance Rank'].astype(int)
data['2023 Payroll Dollars'] = data['2023 Payroll Dollars'].astype(int)

In [ ]:
data[['Name', '2023 Payroll Dollars', 'Payroll Rank']].sort_values(by='Payroll Rank', ascending=True)

,Name,2023 Payroll Dollars,Payroll Rank
37,Jonathan Aranda,720000,1
35,Luken Baker,720000,1
33,Mike Ford,720000,1
4,Bobby Dalbec,720000,1
5,Jared Young,720000,1
28,Alfonso Rivas,720000,1
27,Weston Wilson,720000,1
24,Ryan Noda,720000,1
23,Jake Bauers,720000,1
12,Jon Singleton,720000,1


In [ ]:
data[['Name', 'OE', 'DE', 'Performance Rank']].sort_values(by='Performance Rank', ascending=True)

,Name,OE,DE,Performance Rank
1,Matt Olson,0.954963,0.895479,1
22,Pete Alonso,0.781618,0.913158,2
15,Freddie Freeman,0.806072,0.798799,3
0,Christian Walker,0.711946,0.839099,4
38,Nathaniel Lowe,0.613312,0.910584,5
36,Yandy Diaz,0.707545,0.815752,6
34,Paul Goldschmidt,0.657440,0.838425,7
25,Bryce Harper,0.672930,0.805011,8
8,Josh Naylor,0.641410,0.829447,9
3,Ryan Mountcastle,0.566300,0.869616,10


In [ ]:
# Step 3 is the same: Calculate Cap Efficiency based on Ranks
data['Cap Efficiency Rank-Based'] = data['Performance Rank'] + data['Payroll Rank']

# Calculate a new rank based on 'Cap Efficiency Rank-Based' score, ensuring lower scores get higher ranks
data['Cap Efficiency Rank-Based Rank'] = data['Cap Efficiency Rank-Based'].rank(method='min', ascending=True)

# Sort the DataFrame by the new rank column in ascending order, so the lowest (best) rank is at the top
result_sorted_by_efficiency = data[['Name', 'OE', 'DE', '2023 Payroll Dollars', 'Cap Efficiency Rank-Based', 'Cap Efficiency Rank-Based Rank']].sort_values(by='Cap Efficiency Rank-Based Rank', ascending=True)

# Convert 'Cap Efficiency Rank-Based Rank' to integer to remove decimal points for cleaner presentation
result_sorted_by_efficiency['Cap Efficiency Rank-Based Rank'] = result_sorted_by_efficiency['Cap Efficiency Rank-Based Rank'].astype(int)


In [ ]:
result_sorted_by_efficiency

,Name,OE,DE,2023 Payroll Dollars,Cap Efficiency Rank-Based,Cap Efficiency Rank-Based Rank
24,Ryan Noda,0.515841,0.738498,720000,23,1
3,Ryan Mountcastle,0.566300,0.869616,738400,27,2
14,Nolan Schanuel,0.423396,0.761099,720000,27,2
38,Nathaniel Lowe,0.613312,0.910584,4050000,29,4
33,Mike Ford,0.435727,0.742443,720000,29,4
42,Joey Meneses,0.559540,0.790771,723300,30,6
23,Jake Bauers,0.371425,0.797827,720000,31,7
10,Spencer Torkelson,0.657140,0.674607,723100,32,8
8,Josh Naylor,0.641410,0.829447,3350000,32,8
0,Christian Walker,0.711946,0.839099,6500000,34,10
